# callbacks

> Focus change callback and audio playback JavaScript for the review card stack

In [ ]:
#| default_exp components.callbacks

In [ ]:
#| export
from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.core.models import CardStackUrls
from cjm_fasthtml_card_stack.js.core import generate_card_stack_js

## Focus Change + Audio Audition

When the user navigates to a review card, the focus change callback:
1. Updates the hidden input with the focused segment index
2. Auto-plays the audio for the segment's time range (audition pattern)

Audio playback uses the Web Audio API for precise time-based playback.

In [ ]:
#| export
def _generate_review_focus_change_script(
    focus_input_id:str,  # ID of hidden input for focused segment index
    audio_player_id:str,  # ID of the hidden audio element (for src URL extraction)
    card_stack_id:str,  # ID of the review card stack container
    nav_down_btn_id:str,  # ID of nav_down button for auto-navigate
) -> str:  # JavaScript for focus change with audio playback
    """Generate JS for review focus change handling with Web Audio API playback."""
    return f"""
        // Track the last played segment index to avoid replaying on non-navigation swaps
        window._reviewLastPlayedIndex = null;

        // Web Audio API state
        window._reviewAudioCtx = null;
        window._reviewAudioBuffer = null;
        window._reviewCurrentSource = null;
        window._reviewAudioLoading = false;
        window._reviewAudioError = null;

        // Pending playback (for when audio is still loading)
        window._reviewPendingPlay = null;
        
        // Audio control state (updated by UI controls)
        window._reviewPlaybackSpeed = window._reviewPlaybackSpeed || 1.0;
        window._reviewAutoNavigate = window._reviewAutoNavigate || false;
        
        // Track current segment for replay
        window._reviewCurrentSegment = null;

        // Debug flag — set window.DEBUG_REVIEW_AUDIO = true in browser console to enable
        // Logs: segment index, start/end times, duration, Web Audio API operations
        
        // Note: Pitch changes with speed. True pitch-preserving speed change requires
        // time-stretching algorithms (e.g., SoundTouchJS) not built into Web Audio API.

        // Initialize Web Audio API and load audio buffer
        window.initReviewAudio = async function() {{
            if (window._reviewAudioBuffer || window._reviewAudioLoading) return;
            
            // Get audio URL from the hidden audio element
            var audioEl = document.getElementById('{audio_player_id}');
            if (!audioEl || !audioEl.src) {{
                if (window.DEBUG_REVIEW_AUDIO) {{
                    console.log('[REVIEW_AUDIO] No audio element or src found');
                }}
                return;
            }}
            
            window._reviewAudioLoading = true;
            if (window.DEBUG_REVIEW_AUDIO) {{
                console.log('[REVIEW_AUDIO] Loading audio via Web Audio API:', audioEl.src);
            }}
            
            try {{
                // Create AudioContext
                window._reviewAudioCtx = new (window.AudioContext || window.webkitAudioContext)();
                
                // Fetch and decode audio
                var response = await fetch(audioEl.src);
                var arrayBuffer = await response.arrayBuffer();
                window._reviewAudioBuffer = await window._reviewAudioCtx.decodeAudioData(arrayBuffer);
                
                if (window.DEBUG_REVIEW_AUDIO) {{
                    console.log('[REVIEW_AUDIO] Audio loaded. Duration:', window._reviewAudioBuffer.duration.toFixed(2) + 's',
                        '| Sample rate:', window._reviewAudioBuffer.sampleRate + 'Hz',
                        '| Channels:', window._reviewAudioBuffer.numberOfChannels);
                }}
                
                // Play pending segment if any
                if (window._reviewPendingPlay) {{
                    var pending = window._reviewPendingPlay;
                    window._reviewPendingPlay = null;
                    if (window.DEBUG_REVIEW_AUDIO) {{
                        console.log('[REVIEW_AUDIO] Playing pending segment after load');
                    }}
                    window.playReviewSegment(pending.start, pending.end, pending.indicator);
                }}
            }} catch (e) {{
                window._reviewAudioError = e;
                console.error('[REVIEW_AUDIO] Failed to load audio:', e);
            }} finally {{
                window._reviewAudioLoading = false;
            }}
        }};

        // Stop any currently playing audio
        window.stopReviewAudio = function() {{
            if (window._reviewCurrentSource) {{
                try {{
                    window._reviewCurrentSource.stop();
                }} catch (e) {{
                    // Already stopped
                }}
                window._reviewCurrentSource = null;
            }}
            if (window._reviewPlayTimeout) {{
                clearTimeout(window._reviewPlayTimeout);
                window._reviewPlayTimeout = null;
            }}
        }};

        // Play a specific time range using Web Audio API
        window.playReviewSegment = function(start, end, indicator) {{
            // If audio not loaded yet, trigger load and queue this playback
            if (!window._reviewAudioBuffer) {{
                if (!window._reviewAudioLoading) {{
                    if (window.DEBUG_REVIEW_AUDIO) {{
                        console.log('[REVIEW_AUDIO] Audio not loaded, triggering load...');
                    }}
                    window._reviewPendingPlay = {{ start: start, end: end, indicator: indicator }};
                    window.initReviewAudio();
                }} else {{
                    if (window.DEBUG_REVIEW_AUDIO) {{
                        console.log('[REVIEW_AUDIO] Audio loading, queueing playback...');
                    }}
                    window._reviewPendingPlay = {{ start: start, end: end, indicator: indicator }};
                }}
                return;
            }}
            
            // Resume AudioContext if suspended (browser autoplay policy)
            if (window._reviewAudioCtx.state === 'suspended') {{
                window._reviewAudioCtx.resume();
            }}
            
            // Stop previous playback
            window.stopReviewAudio();
            
            // Clamp values
            var duration = end - start;
            if (start < 0) start = 0;
            if (end > window._reviewAudioBuffer.duration) end = window._reviewAudioBuffer.duration;
            if (duration <= 0) return;
            
            // Store current segment for replay
            window._reviewCurrentSegment = {{ start: start, end: end, indicator: indicator }};
            
            // Get playback speed
            var speed = window._reviewPlaybackSpeed || 1.0;
            
            // Create buffer source with playback rate
            var source = window._reviewAudioCtx.createBufferSource();
            source.buffer = window._reviewAudioBuffer;
            source.playbackRate.value = speed;
            source.connect(window._reviewAudioCtx.destination);
            window._reviewCurrentSource = source;
            
            // Play segment (start at context time 0, offset into buffer at 'start', for 'duration')
            source.start(0, start, duration);
            
            if (window.DEBUG_REVIEW_AUDIO) {{
                console.log('[REVIEW_AUDIO] Playing segment:', start.toFixed(2) + 's ->', end.toFixed(2) + 's',
                    '| duration:', duration.toFixed(2) + 's',
                    '| speed:', speed + 'x');
            }}
            
            // Calculate actual playback time accounting for speed
            var playbackTime = (duration / speed) * 1000;
            
            // Hide indicator and handle auto-navigate after playback completes
            window._reviewPlayTimeout = setTimeout(function() {{
                if (indicator) {{
                    indicator.classList.add('invisible');
                    indicator.classList.remove('visible');
                }}
                if (window.DEBUG_REVIEW_AUDIO) {{
                    console.log('[REVIEW_AUDIO] Playback completed');
                }}
                
                // Auto-navigate to next segment if enabled
                if (window._reviewAutoNavigate) {{
                    var navDownBtn = document.getElementById('{nav_down_btn_id}');
                    if (navDownBtn) {{
                        if (window.DEBUG_REVIEW_AUDIO) {{
                            console.log('[REVIEW_AUDIO] Auto-navigating to next segment');
                        }}
                        navDownBtn.click();
                    }}
                }}
            }}, playbackTime);
        }};
        
        // Replay the current segment (called by Space key or replay button)
        window.replayReviewSegment = function() {{
            var seg = window._reviewCurrentSegment;
            if (!seg) {{
                if (window.DEBUG_REVIEW_AUDIO) {{
                    console.log('[REVIEW_AUDIO] No current segment to replay');
                }}
                return;
            }}
            
            if (window.DEBUG_REVIEW_AUDIO) {{
                console.log('[REVIEW_AUDIO] Replaying current segment');
            }}
            
            // Force replay by resetting last played index
            window._reviewLastPlayedIndex = null;
            
            // Show indicator
            if (seg.indicator) {{
                seg.indicator.classList.remove('invisible');
                seg.indicator.classList.add('visible');
            }}
            
            // Play the segment
            window.playReviewSegment(seg.start, seg.end, seg.indicator);
        }};

        // Called when card focus changes in the review zone
        window.onReviewFocusChange = function(item, index, zoneId) {{
            // Update hidden input with focused segment index
            var input = document.getElementById('{focus_input_id}');
            if (input && item) {{
                input.value = item.dataset.segmentIndex || index;
            }}

            // Check if this is a new card (navigation) vs same card
            var currentIndex = parseInt(item.dataset.segmentIndex || index, 10);
            if (window._reviewLastPlayedIndex === currentIndex) {{
                if (window.DEBUG_REVIEW_AUDIO) {{
                    console.log('[REVIEW_AUDIO] Same card, skipping replay. Index:', currentIndex);
                }}
                return;
            }}
            window._reviewLastPlayedIndex = currentIndex;

            // Hide all playing indicators
            document.querySelectorAll('.review-playing-indicator').forEach(function(el) {{
                el.classList.add('invisible');
                el.classList.remove('visible');
            }});

            if (!item) return;

            var start = parseFloat(item.dataset.startTime || 0);
            var end = parseFloat(item.dataset.endTime || 0);
            
            if (window.DEBUG_REVIEW_AUDIO) {{
                console.log('[REVIEW_AUDIO] Segment', currentIndex, '| start:', start.toFixed(2) + 's', '| end:', end.toFixed(2) + 's');
            }}
            
            if (end <= start) return;

            // Show playing indicator on current card
            var indicator = item.querySelector('.review-playing-indicator');
            if (indicator) {{
                indicator.classList.remove('invisible');
                indicator.classList.add('visible');
            }}

            // Play using Web Audio API (will auto-init if needed)
            window.playReviewSegment(start, end, indicator);
        }};

        // HTMX afterSettle listener — trigger focus change callback after navigation swaps
        (function() {{
            var handlerKey = '_reviewFocusSettleHandler';
            
            if (window[handlerKey]) {{
                document.body.removeEventListener('htmx:afterSettle', window[handlerKey]);
            }}
            
            function afterSettleHandler(evt) {{
                var target = evt.detail.target;
                if (!target) return;
                
                var cardStack = document.getElementById('{card_stack_id}');
                if (!cardStack) return;
                
                var isReviewSwap = (
                    target.id === '{card_stack_id}' ||
                    cardStack.contains(target)
                );
                if (!isReviewSwap) return;
                
                var state = window.kbNav && window.kbNav.getState && window.kbNav.getState();
                var isReviewActive = state && state.activeZoneId === '{card_stack_id}';
                if (!isReviewActive) return;
                
                var focusedCard = cardStack.querySelector('[data-card-role="focused"]');
                if (focusedCard && window.onReviewFocusChange) {{
                    var index = parseInt(focusedCard.dataset.segmentIndex || '0', 10);
                    window.onReviewFocusChange(focusedCard, index, '{card_stack_id}');
                }}
            }}
            
            window[handlerKey] = afterSettleHandler;
            document.body.addEventListener('htmx:afterSettle', afterSettleHandler);
        }})();
    """

## Combined Callbacks Script

Wraps the card stack library's JS generator with review-specific focus change callback.

In [ ]:
#| export
def generate_review_callbacks_script(
    ids:CardStackHtmlIds,  # Card stack HTML IDs
    button_ids:CardStackButtonIds,  # Card stack button IDs
    config:CardStackConfig,  # Card stack configuration
    urls:CardStackUrls,  # Card stack URL bundle
    container_id:str,  # ID of the review container (parent of card stack)
    focus_input_id:str,  # ID of hidden input for focused segment index
    audio_player_id:str,  # ID of the hidden audio element
) -> any:  # Script element with all JavaScript callbacks
    """Generate JavaScript for review card stack with audio audition."""
    review_scripts = (
        _generate_review_focus_change_script(
            focus_input_id=focus_input_id,
            audio_player_id=audio_player_id,
            card_stack_id=ids.card_stack,
            nav_down_btn_id=button_ids.nav_down,
        ),
    )

    return generate_card_stack_js(
        ids=ids,
        button_ids=button_ids,
        config=config,
        urls=urls,
        container_id=container_id,
        extra_scripts=review_scripts
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()